In [1]:
from pathlib import Path
import sys

__HOME__ = Path("/nas/home/minhpham/workspace/kb-data-cleaning")

input_file = __HOME__ / "data/test/oecd/oecd.csv"

if str(__HOME__ / "kbclean") not in sys.path:
    sys.path.append(str(__HOME__ / "kbclean"))

In [2]:
import pandas as pd
import numpy as np

raw_df = pd.read_csv(input_file, header=None, index_col=None)

df = raw_df.iloc[4:133, 3:10]
df = df.transpose()
df.columns = raw_df.iloc[4:133, 1]

df = df.applymap(lambda x: x.encode("ascii", "ignore").decode("ascii"))
df = df.replace("", np.nan).dropna(how='all', axis=1)

df.head(5)

1,GDP per capita,Gross national income (GNI) per capita,Household disposable income,Real GDP growth,Net saving rate in household disposable income,Gross fixed capital formation,"Agriculture, forestry, fishing",Industry including energy,Construction,"Trade, repairs, transport, accomm., food services",...,Unemployment rate in population of native-born women,Unemployment rate in population of foreign-born women,Life expectancy at birth,Life expectancy at birth: men,Life expectancy at birth: women,Infant mortality,"Overweight or obese, % of population aged 15 and over",Suicide Rates,Goods transport,Passenger transport
3,41 450,41 868,-0.5,1.8,6.5,4.2,0.7,17.3,5.7,20.2,...,6.0,14.6,80.7 |,78.0 |,83.3 |,3.4,..,18.1,50 506 e,138 643
4,42 585,43 627,0.2,0.2,5.7,0.2,0.9,16.8,5.7,20.0,...,5.9,15.9,80.5,77.8,83.1,3.8,..,17.4,..,..
5,43 746,44 467,0.2,0.2,5.1,-1.5,0.8,16.7,5.6,19.8,...,6.8,16.0,80.7,78.1,83.2,3.5,..,16.2,..,132 125
6,44 720,45 029,1.1,1.3,5.1,5.8,0.7,16.5,5.5,19.7,...,6.5,16.3,81.4,78.8,83.9,3.4,51.0,16.1,..,134 954 e
7,45 739,45 480,0.4,1.7,4.3,2.7,0.8,16.7,5.3,19.7,...,6.2,16.0,81.1,78.7,83.4,3.3,..,15.8,..,132 573


In [3]:
from argparse import Namespace

hparams = {"batch_size": 1000}
hparams = Namespace(**hparams)

In [4]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from models.language_modeler import BertLanguageModel, RNNLanguageModel

tokenizer = ByteLevelBPETokenizer(str(__HOME__ / "webtables-vocab.json"), str(__HOME__ / "webtables-merges.txt"))

tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")), ("<s>", tokenizer.token_to_id("<s>")),
)
hparams.vocab_size = tokenizer.get_vocab_size()
tokenizer.enable_truncation(max_length=100)

error_model = BertLanguageModel.load_from_checkpoint(
    checkpoint_path=str(__HOME__ / "models/bert.ckpt"), tokenizer=tokenizer
)

In [55]:
import regex as re

regex_dict = {"digit":r"[-+]?[0-9]+", "lower":r"[a-z]+", "upper":r"[A-Z]+", "whitespace": r" "}

def featurize(str_):
    feature_dict = {}
    count = 0
    while str_:
        count += 1
        for name, pattern in regex_dict.items():
            match = re.match(f"^{pattern}", str_)
            if match:
                feature_dict[f"{name}_{count}"] = 1
#                 feature_dict[f"{match.group()}_{count}"] = 1
                str_ = str_[match.end():]
                break
        else:
            feature_dict[f"{str_[0]}_{count}"] = 1
            str_ = str_[1:]
    return feature_dict


feature_df = df.applymap(lambda x: featurize(x))
feature_df

1,GDP per capita,Gross national income (GNI) per capita,Household disposable income,Real GDP growth,Net saving rate in household disposable income,Gross fixed capital formation,"Agriculture, forestry, fishing",Industry including energy,Construction,"Trade, repairs, transport, accomm., food services",...,Unemployment rate in population of native-born women,Unemployment rate in population of foreign-born women,Life expectancy at birth,Life expectancy at birth: men,Life expectancy at birth: women,Infant mortality,"Overweight or obese, % of population aged 15 and over",Suicide Rates,Goods transport,Passenger transport
3,"{'digit_1': 1, 'whitespace_2': 1, 'digit_3': 1}","{'digit_1': 1, 'whitespace_2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}",...,"{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1, 'whites...","{'digit_1': 1, '._2': 1, 'digit_3': 1, 'whites...","{'digit_1': 1, '._2': 1, 'digit_3': 1, 'whites...","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'whitespace_1': 1, '._2': 1, '._3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, 'whitespace_2': 1, 'digit_3': 1...","{'digit_1': 1, 'whitespace_2': 1, 'digit_3': 1}"
4,"{'digit_1': 1, 'whitespace_2': 1, 'digit_3': 1}","{'digit_1': 1, 'whitespace_2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}",...,"{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'whitespace_1': 1, '._2': 1, '._3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'whitespace_1': 1, '._2': 1, '._3': 1}","{'whitespace_1': 1, '._2': 1, '._3': 1}"
5,"{'digit_1': 1, 'whitespace_2': 1, 'digit_3': 1}","{'digit_1': 1, 'whitespace_2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}",...,"{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'whitespace_1': 1, '._2': 1, '._3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'whitespace_1': 1, '._2': 1, '._3': 1}","{'digit_1': 1, 'whitespace_2': 1, 'digit_3': 1}"
6,"{'digit_1': 1, 'whitespace_2': 1, 'digit_3': 1}","{'digit_1': 1, 'whitespace_2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}",...,"{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'digit_1': 1, '._2': 1, 'digit_3': 1}","{'whitespace_1'

In [56]:
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.feature_extraction import DictVectorizer

result_df = df.copy()
detector = IsolationForest()
dict_vectorizer = DictVectorizer()
feature_dicts = []
values = []
for column in df.columns:
    values.extend(df[column].values.tolist())
    feature_dicts.extend(feature_df[column].values.tolist())
feature_vecs = dict_vectorizer.fit_transform(feature_dicts)
#     outliers = detector.fit_predict(feature_vecs)
#     if all(outliers == -1):
#         outliers = -outliers

#     result_df[column] = pd.Series(zip(outliers, values), index=result_df.index).apply(lambda x: f"[[{x[1]}]]" if x[0] == -1 else f"{x[1]}")

In [57]:
outliers = detector.fit_predict(feature_vecs)

list(filter(lambda x:x[0] == -1, zip(outliers, values)))

[(-1, '2.3 |'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, '110'),
 (-1, '104'),
 (-1, '109'),
 (-1, '109'),
 (-1, '100'),
 (-1, '100'),
 (-1, '101'),
 (-1, '3 119.8'),
 (-1, '3 364.7'),
 (-1, '3 505.0'),
 (-1, '3 398.2'),
 (-1, '3 664.3'),
 (-1, '3 916.0'),
 (-1, '4 129.0'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, ' ..'),
 (-1, '17'),
 (-1, '18'),
 (-1, '16'),
 (-1, '20'),
 (-1, '18'),
 (-1, '17'),
 (-1, '17'),
 (-1, '3'),
 (-1, '4'),
 (-1, '6'),
 (-1, '5'),
 (-1, '4'),
 (-1, '8'),
 (-1, ' ..'),
 (-1, '0 e'),
 (-1, '0 e'),
 (-1, '0 e'),
 (-1, '0 e'),
 (-1, '0 e'),
 (-1, '0'),
 (-1, ' ..'),
 (-1, '454'),
 (-1, '446'),
 (-1, '436'),
 (-1, '424'),
 (-1, '411'),
 (-1, '418'),
 (-1, '408'),
 (-1, '93'),
 (-1, '92'),
 (-1, '94'),
 (-1, '87'),
 (-1, '93'),
 (-1, '92'),
 (-1, ' ..'),
 (-1, '35'),
 (-1, '35'),
 (-1, '36'),
 (-1, '37'),
 (-1, '37'),
 (-1, '3

In [15]:
result_df.to_csv(__HOME__ / "results/oecd_detected.csv")

In [16]:
result_df

1,GDP per capita,Gross national income (GNI) per capita,Household disposable income,Real GDP growth,Net saving rate in household disposable income,Gross fixed capital formation,"Agriculture, forestry, fishing",Industry including energy,Construction,"Trade, repairs, transport, accomm., food services",...,Unemployment rate in population of native-born women,Unemployment rate in population of foreign-born women,Life expectancy at birth,Life expectancy at birth: men,Life expectancy at birth: women,Infant mortality,"Overweight or obese, % of population aged 15 and over",Suicide Rates,Goods transport,Passenger transport
3,41 450,41 868,[[-0.5]],1.8,6.5,4.2,0.7,17.3,5.7,20.2,...,6.0,14.6,[[80.7 |]],[[78.0 |]],[[83.3 |]],3.4,..,18.1,[[50 506 e]],138 643
4,42 585,43 627,0.2,0.2,5.7,0.2,0.9,16.8,5.7,20.0,...,5.9,15.9,80.5,77.8,83.1,3.8,..,17.4,..,[[ ..]]
5,43 746,44 467,0.2,0.2,5.1,[[-1.5]],0.8,16.7,5.6,19.8,...,6.8,16.0,80.7,78.1,83.2,3.5,..,16.2,..,132 125
6,44 720,45 029,1.1,1.3,5.1,5.8,0.7,16.5,5.5,19.7,...,6.5,16.3,81.4,78.8,83.9,3.4,[[51.0]],16.1,..,[[134 954 e]]
7,45 739,45 480,0.4,1.7,4.3,2.7,0.8,16.7,5.3,19.7,...,6.2,16.0,81.1,78.7,83.4,3.3,..,15.8,..,132 573
8,47 366,47 420,1.3,1.5,3.9,3.8,0.7,16.6,5.3,19.7,...,6.0,15.5,81.5,79.0,84.0,3.2,..,15.9,..,[[ ..]]
9,49 526,50 109,1.2,1.7,4.0,1.8,0.7,16.7,5.2,19.5,...,5.7,13.8,81.6,79.2,83.9,3.6,..,[[ ..]],..,[[ ..]]
